# Co-Simulation

In this example, we simulate a basic energy system of a small datacenter equipped with on-site solar power and a simple battery.

In [1]:
import pandas as pd

from vessim.actor import ComputingSystem, Generator
from vessim.controller import Monitor
from vessim.cosim import Environment
from vessim.power_meter import MockPowerMeter
from vessim.signal import HistoricalSignal
from vessim.storage import SimpleBattery

# Hotfix to execute asyncio in Jupyter
import nest_asyncio
nest_asyncio.apply()

We define an `Environment` with a single `Microgird` that comprises

- A simulated `ComputingSystem` with two servers that consistently draw 200W and 250W, respectively. The datacenter has a [power usage effectiveness](https://en.wikipedia.org/wiki/Power_usage_effectiveness) of 1.6, resulting in a total power demand of 1.6 * (200 W + 250 W) = 720 W.
- A `Generator` which represents a solar panel in Berlin, modeled according to the dataset `solcast2022_global` that comes with Vessim (see "Signal" example).
- A `SimpleBattery` with a usable capacity of 100 Wh which is charged with 1Wh.
- A `Monitor` which periodically stores the microgrid state and eventually writes it to a CSV file.

In [2]:
environment = Environment(sim_start="2020-06-11 00:00:00")

monitor = Monitor()  # stores simulation result on each step
environment.add_microgrid(
    actors=[
        ComputingSystem(power_meters=[MockPowerMeter(p=200), MockPowerMeter(p=250)], pue=1.6),
        Generator(signal=HistoricalSignal.from_dataset("solcast2022_global"), column="Berlin"),
    ],
    controllers=[monitor],
    storage=SimpleBattery(capacity=100, charge_level=100),
    step_size=60,  # global step size (can be overridden by actors or controllers)
)

environment.run(until=24 * 3600)  # 24h
monitor.to_csv("result.csv")

2024-03-13 16:21:00.788 | INFO     | mosaik.scenario:start:280 - Starting "Actor" as "Actor-0" ...
2024-03-13 16:21:00.789 | INFO     | mosaik.scenario:start:280 - Starting "Actor" as "Actor-1" ...
2024-03-13 16:21:00.790 | INFO     | mosaik.scenario:start:280 - Starting "Grid" as "Grid-0" ...
2024-03-13 16:21:00.791 | INFO     | mosaik.scenario:start:280 - Starting "Controller" as "Controller-0" ...
2024-03-13 16:21:00.794 | INFO     | mosaik.scenario:run:598 - Starting simulation.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86400/86400 [00:00<00:00, 117148.08steps/s]
2024-03-13 16:21:01.542 | INFO     | mosaik.scenario:run:646 - Simulation finished successfully.


For example, we can observe that at 10pm, the battery is fully drained and there is also no solar power production at night.
The resulting $p_{delta}$ of -720 
W has to be drawn from the public grid.

In [3]:
monitor.monitor_log[pd.to_datetime('2020-06-11 22:00:00')]["p_delta"]

-720.0